<a href="https://colab.research.google.com/github/yejinee/Recommendation_System/blob/main/Best_Seller_Recommendation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
[ CH2. 기본적인 추천 시스템 ] 
Data 설명 (MovieLens)
- 사용자들이 영화에 대해 평가한 데이터
- 각 영화를 1점(최악) ~ 5점(최고) 사이의 점수로 평가
- 전체 평점의 갯수 2700백만개이나 이 중 일부를 추출하여 100k, 200M만 사용
- 3가지 파일로 구성   
(u.user: 사용자 데이터, u.item: 영화 데이터, u.data: 영화평가파일)
"""

'\n[ CH2. 기본적인 추천 시스템 ] \nData 설명 (MovieLens)\n- 사용자들이 영화에 대해 평가한 데이터\n- 각 영화를 1점(최악) ~ 5점(최고) 사이의 점수로 평가\n- 전체 평점의 갯수 2700백만개이나 이 중 일부를 추출하여 100k, 200M만 사용\n- 3가지 파일로 구성   \n(u.user: 사용자 데이터, u.item: 영화 데이터, u.data: 영화평가파일)\n'

In [9]:
from google.colab import files
users=files.upload()

Saving user.txt to user.txt


In [10]:
import io
import pandas as pd

In [14]:
u_cols=['user_id','age','sex','occipation','zip_code']

In [25]:
users=pd.read_csv('user.txt',sep='|',names=u_cols)
users=users.set_index('user_id')
users.head()

,age,sex,occipation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [18]:
# u.item file을 Dataframe으로 읽기 (영화 정보 )
data=files.upload()

Saving u.item to u.item


In [22]:
cols=['movie_id','title','release date','video release date','IMDB URL','unknown','Action','Adventure','Animation','children\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romancce','Sci-Fi','Thriller','War','Western']

In [24]:
movies=pd.read_csv('u.item',sep='|',names=cols,encoding='latin=1')
movies=movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romancce,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [26]:
# u.data file을 DataFrame으로 읽기
data1=files.upload()

Saving u.data to u.data


In [27]:
r_cols=['user_id','movie_id','rating','timestamp']

In [29]:
ratings=pd.read_csv('u.data',sep='\t',names=r_cols,encoding='latin=1')
ratings=ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


In [30]:
"""
[ 인기제품 방식 ]
- 각 영화에 대한 평가(평점)을 평균해서 평균값이 높은 것을 순서대로 추천하는 방식 
"""

'\n[ 인기제품 방식 ]\n- 각 영화에 대한 평가(평점)을 평균해서 평균값이 높은 것을 순서대로 추천하는 방식 \n'

In [32]:
# Best-seller Recommendation
def rec_bestmovie(n_items):
  # 1. 영화의 평점 평균인 movie_mean을 내림차순으로 정렬 + 상위 n_times개 만큼 movie_sort에 저장
  movie_sort=movie_mean.sort_values(ascending=False)[:n_items]   # (movie_mean이 Series형식이므로 movie_sort도 Series형식임)
  # 2. n_times개 영화에 대한 정보를 저장 
  recom_movies=movies.loc[movie_sort.index] 
  # 선정된 영화들에서 title만 골라서 반환
  recommendations=recom_movies['title']
  return recommendations

movie_mean=ratings.groupby(['movie_id'])['rating'].mean() # movie_id가 같은 영화 별로 rating의 평균 구함 (pandas의 Series형식)
rec_bestmovie(5)


movie_id
1293                                      Star Kid (1997)
1467                 Saint of Fort Washington, The (1993)
1653    Entertaining Angels: The Dorothy Day Story (1996)
814                         Great Day in Harlem, A (1994)
1122                       They Made Me a Criminal (1939)
Name: title, dtype: object

In [33]:
"""
[ 정확도 계산하기 ]
- test set을 미리 분리하고 train set으로 예측값 게산 
- 예측값을 test set의 실제값과 비교해 정확도 계산 
- RMSE 방법을 사용 
"""

'\n[ 정확도 계산하기 ]\n- test set을 미리 분리하고 train set으로 예측값 게산 \n- 예측값을 test set의 실제값과 비교해 정확도 계산 \n- RMSE 방법을 사용 \n'

In [34]:
import numpy as np
# y_true: 실제값의 벡터, y_pred: 예측값의 벡터 
def RMSE(y_true,y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

In [36]:
# Best-seller방식으로 구한 예측값의 RMSE 계산
rmse=[]
for user in set(ratings.index): #ratings의 각 user에 대해 roop
  y_true=ratings.loc[user]['rating'] # 사용자가 평가한 모든 영화의 평점 저장
  y_pred=movie_mean[ratings.loc[user]['movie_id']] # 해당 사용자가 평가한 영화의 평점평균 저장
  accuracy=RMSE(y_true,y_pred) # 사용자 1명에 대한 RMSE 
  rmse.append(accuracy)
print(np.mean(rmse)) # 전체 사용자의 RMSE평균 

0.996007224010567
